# Классификация imdb

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds

import pandas as pd

from sklearn.preprocessing import StandardScaler, MinMaxScaler
print(tf.__version__)

In [0]:
tfds.builder('imdb_reviews').info

## Импортируем IMDB датасет

In [0]:
imdb = keras.datasets.imdb

In [0]:
(train_vec, train_labels), (test_vec, test_labels) = imdb.load_data()

Датасет содержит следующие классы:
0 - Negative
1 - Positive

Каждому классу обозначенному цифрой мы можем присвоить текстовое значение -

In [0]:
class_names = ['Negative', 'Positive']

## Анализ датасета 



Проанализируем тестовую выборку:

In [0]:
print(len(test_vec[55]))
print(len(test_vec[33]))

In [0]:
df = pd.DataFrame(train_vec)

In [0]:
df

Видно, что вектора в датасете разной длины. Так не годится. Следует выровнять. Попробуем дополнить нулями/урезать

In [0]:
df[0].apply(lambda x : len(x)).mean

In [0]:
#Средняя длина вектора - 218. Возьмем 256 для ровного счета

## Preprocess the data

Приведем все векторы к одной длине


In [0]:
from keras.preprocessing.sequence import pad_sequences

In [0]:
meanlen = 256

In [0]:
df_pad = pad_sequences(df, maxlen=meanlen)

In [0]:
scaler = StandardScaler()

df_norm = train_images.copy()
df_norm = scaler.fit_transform(df_norm)

In [0]:
plt.figure()
plt.imshow(train_images[0])
plt.colorbar()
plt.grid(False)
plt.show()

Для процесса обучения нейронной сети нам важно перевести данные из диапазона от 0 до 255 в диапазон от 0 до 1:

In [0]:
train_images = train_images / 255.0

test_images = test_images / 255.0

Посмотрим первые 25 изображений:

In [0]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[train_labels[i]])
plt.show()

## Построение модели

Построение нейронной сети подразумевает конфигурацию ее слоев и последующую компиляцию.

### Определение слоев

Давайте создадим 3 слоя нейронной сети с помощью функционала Keras.layers

In [0]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(10)
])

Первый слой, `tf.keras.layers.Flatten`, трасформирует двумерный массив на входе в одномерный массив.

Получившиеся 784(28 x 28) входных нейрона присоединяем к полносвязному слою из 128 нейронов , которые будут использовать функцию активации relu. В выходном слое будет 10 нейронов, по числу классов, которые он должен предсказывать. В нем будет использоваться функция активации softmax и он будет давать предсказание от 0 до 1, где 1 это стопроцентная вероятность.

### Компиляция модели

Давайте вспомним ключевые понятия, которые нам понадобяться при компиляции:

* *Loss function* — меряет как точно работает нейросеть.
* *Optimizer* — определяет способ корректировки весов.
* *Metrics* — определяет за какие характеристики будут отражаться в процессе обучения.

In [0]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

## Тренировка модели

Здесь все стандартно - данные передаются в нейросеть и сопоставляются изображения и лейблы.



### Передача данных в модель

Команда непосредственно запускающая процесс обучения называется - `model.fit`:

In [0]:
model.fit(train_images, train_labels, epochs=3)

In [0]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

print('\nTest accuracy:', test_acc)

В выводе выше мы следим за точностью в процессе обучения, проверяем точность на тестовых даыннх и меняем параметры нейросети если точность на тестовых данных нас не устраивает.

### Предсказания нейросети

Команды ниже позволяют проверить работу натренированной ранее нейросети - 

In [0]:
probability_model = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])

In [0]:
predictions = probability_model.predict(test_images)

In [0]:
predictions[0]

In [0]:
np.argmax(predictions[0])

In [0]:
test_labels[0]

In [0]:
def plot_image(i, predictions_array, true_label, img):
  predictions_array, true_label, img = predictions_array, true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img, cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  predictions_array, true_label = predictions_array, true_label[i]
  plt.grid(False)
  plt.xticks(range(10))
  plt.yticks([])
  thisplot = plt.bar(range(10), predictions_array, color="#777777")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)

  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')

### Проверка предсказаний

Matplotlib нам дает возможность посмотреть наше предсказание графически:

In [0]:
i = 0
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions[i], test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions[i],  test_labels)
plt.show()

In [0]:
i = 12
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions[i], test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions[i],  test_labels)
plt.show()

Давайте сделаем еще несколько предсказаний - 

In [0]:

num_rows = 5
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_image(i, predictions[i], test_labels, test_images)
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_value_array(i, predictions[i], test_labels)
plt.tight_layout()
plt.show()

## Использование полученной модели

Давайте возьмем одно изображение из тестовой выборке и посмотрим предсказание нейронной сети - 

In [0]:

img1 = test_images[1]

print(img1.shape)

In [0]:
# Add the image to a batch where it's the only member.
img = (np.expand_dims(img1,0))

print(img.shape)

In [0]:
predictions_single = probability_model.predict(img)

print(predictions_single)

In [0]:
plot_value_array(1, predictions_single[0], test_labels)
_ = plt.xticks(range(10), class_names, rotation=45)

`keras.Model.predict` возвращает список списков — по одному списку для каждого предсказания в батче. Нам нужны предсказания только для одного изображения:

In [0]:
np.argmax(predictions_single[0])

Нейросеть при хорошо подобранных параметрах должна была выдать корректное предсказание

In [0]:
model1 = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(10)
])

model2 = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dense(10)
])

model3 = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dense(10)
])

model1.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model2.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model3.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

hist1=model1.fit(train_images, train_labels, epochs=30)
hist2=model2.fit(train_images, train_labels, epochs=30)
hist3=model3.fit(train_images, train_labels, epochs=30)



In [0]:
plt.plot(hist1.history['accuracy'],label='model1 acc')
plt.plot(hist2.history['accuracy'],label='model2 acc')
plt.plot(hist3.history['accuracy'],label ='model3 acc')
plt.legend()
plt.show()

In [0]:
# Save the model to disk.
model1.save_weights('model1.h5')
model2.save_weights('model2.h5')
model3.save_weights('model3.h5')
# Load the model from disk later using:
# model.load_weights('model.h5')

In [0]:
input1 = keras.layers.Input(shape=(28, 28))
x1= keras.layers.Flatten()(input1)
x1 = keras.layers.Dense(128, activation='relu')(x1)
x1 =keras.layers.Dense(10)(x1)

model11 =keras.models.Model(inputs=input1,outputs=x1)


x2= keras.layers.Flatten()(input1)
x2 = keras.layers.Dense(256, activation='relu')(x2)
x2=keras.layers.Dense(10)(x2)

model22 =keras.models.Model(inputs=input1,outputs=x2)

x3= keras.layers.Flatten()(input1)
x3 = keras.layers.Dense(256, activation='relu')(x3)
x3=keras.layers.Dense(10)(x3)


model33 =keras.models.Model(inputs=input1,outputs=x3)
model11.load_weights('model1.h5')
model22.load_weights('model2.h5')
model33.load_weights('model3.h5')

out_all = keras.layers.Average()([model11.output,model22.output,model33.output])
out_all = keras.layers.Softmax()(out_all)
model_all=keras.models.Model(inputs =[input1],outputs =out_all)

In [0]:
model_all.summary()

In [0]:
model_all.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [0]:
predictions_single=model_all.predict(img)

print(predictions_single)

In [0]:
plot_value_array(1, predictions_single[0], test_labels)
_ = plt.xticks(range(10), class_names, rotation=45)

In [0]:
test_loss, test_acc = model_all.evaluate(test_images,  test_labels, verbose=2)

In [0]:
from tensorflow.keras.utils import plot_model

In [0]:
plot_model(model_all,to_file='new_model-all.png')

## стек сетей

In [0]:
from sklearn.metrics import confusion_matrix 

In [0]:
# стек сетей#
y_pred = model_all.predict(test_images)

y_pred_arg = np.argmax(y_pred, axis=1)




In [0]:
test_labels[0]

In [0]:
y_pred_arg.shape

In [0]:
y_pred_arg[0]

In [0]:
confusion_matrix(test_labels,  y_pred_arg)

Слои для объединения

In [0]:
first_input = keras.layers.Input(shape=(28,28 ))
x11= keras.layers.Flatten()(first_input)
first_dense = keras.layers.Dense(1, )(x11)

second_input = keras.layers.Input(shape=(28,28 ))
x22= keras.layers.Flatten()(second_input)
second_dense = keras.layers.Dense(1, )(x22)

merge_one = keras.layers.concatenate([first_dense, second_dense])

third_input = keras.layers.Input(shape=(28,28 ))
x33= keras.layers.Flatten()(third_input)
merge_two = keras.layers.concatenate([merge_one, x33])
merge_two=keras.layers.Dense(10)(merge_two)
merge_two = keras.layers.Softmax()(merge_two)

model_stek = keras.models.Model(inputs=[first_input, second_input, third_input], outputs=merge_two)
ada_grad = keras.optimizers.Adagrad(lr=0.1, epsilon=1e-08, decay=0.0)
model_stek .compile(optimizer=ada_grad, loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [0]:
plot_model(model_stek ,to_file='new_model_stek .png')

In [0]:
# merge samples, two input must be same shape
inp1 =  keras.layers.Input(shape=(10,32))
inp2 =  keras.layers.Input(shape=(10,32))
cc1 =  keras.layers.concatenate([inp1, inp2],axis=0) # Merge data must same row column
output =  keras.layers.Dense(30, activation='relu')(cc1)
model_3 =  keras.models.Model(inputs=[inp1, inp2], outputs=output)
model_3.summary()



In [0]:
# merge row must same column size
inp1 =  keras.layers.Input(shape=(20,10))
inp2 =  keras.layers.Input(shape=(32,10))
cc1 =  keras.layers.concatenate([inp1, inp2],axis=1)
output =  keras.layers.Dense(30, activation='relu')(cc1)
model_4 =  keras.models.Model(inputs=[inp1, inp2], outputs=output)
model_4.summary()



In [0]:
# merge column must same row size
inp1 =  keras.layers.Input(shape=(10,20))
inp2 =  keras.layers.Input(shape=(10,32))
cc1 =  keras.layers.concatenate([inp1, inp2],axis=2)
output =  keras.layers.Dense(30, activation='relu')(cc1)
model_5 =  keras.models.Model(inputs=[inp1, inp2], outputs=output)
model.summary()

In [0]:
inp1 =  keras.layers.Input(shape=(10,32))
inp2 =  keras.layers.Input(shape=(10,32))
cc1 =  keras.layers.Add()([inp1, inp2]) # Merge data must same row column
output =  keras.layers.Dense(30, activation='relu')(cc1)
model_31 =  keras.models.Model(inputs=[inp1, inp2], outputs=output)
model_31.summary()


In [0]:
inp1 =  keras.layers.Input(shape=(10,32))
inp2 =  keras.layers.Input(shape=(10,32))
cc1 =  keras.layers.Multiply()([inp1, inp2]) # Merge data must same row column
output =  keras.layers.Dense(30, activation='relu')(cc1)
model_32 =  keras.models.Model(inputs=[inp1, inp2], outputs=output)
model_32.summary()


In [0]:
inp1 =  keras.layers.Input(shape=(10,32))
inp2 =  keras.layers.Input(shape=(10,32))
cc1 =  keras.layers.Subtract()([inp1, inp2]) # Merge data must same row column
output =  keras.layers.Dense(30, activation='relu')(cc1)
model_3 =  keras.models.Model(inputs=[inp1, inp2], outputs=output)
model_3.summary()


In [0]:
def myFunc(C):
  
  return C[0]*C[1]

In [0]:

inp1 =  keras.layers.Input(shape=(10,32))
inp2 =  keras.layers.Input(shape=(10,32))
cc1 =  keras.layers.Subtract()([inp1, inp2]) 
cross2 = keras.layers.Lambda(myFunc)([cc1,cc1])
output =  keras.layers.Dense(30, activation='relu')(cross2)
model_35 =  keras.models.Model(inputs=[inp1, inp2], outputs=output)
model_35.summary()

## Практическое задание

<ol>
    <li>Попробуйте обучить нейронную сеть на TensorFlow 2 на любом датасете imdb_reviews. 
        Опишите в комментарии к уроку - какой результата вы добились от нейросети? Что помогло вам улучшить ее точность?<br><br>
    </li>
    <li>*2. Поработайте с документацией TensorFlow 2. Найдите полезные команды не разобранные на уроке.</li>
    
    
</ol>

## Дополнительные материалы

<ol>
    <li>www.tensorflow.org/api_docs</li>
</ol>

## Используемая литература 

Для подготовки данного методического пособия были использованы следующие ресурсы:
<ol>
    <li>https://www.tensorflow.org/</li>
    <li>https://www.tensorflow.org/tutorials/keras/classification</li>
    <li>Singh P., Manure A. - Learn TensorFlow 2.0 - 2020</li>
    <li>Шакла Н. — Машинное обучение и TensorFlow 2019</li>
    <li>Википедия</li>
    
</ol>